In [1]:
import importlib
import utils.utilityFunctions as utilityFunctions
importlib.reload(utilityFunctions)

ModuleNotFoundError: No module named 'utils'

In [1]:
from mlflow.tracking import MlflowClient
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

client = MlflowClient()
model_name = "fake-news-model"

# Alle Versionen abrufen
versions = client.search_model_versions(f"name='{model_name}'")

versions

[<ModelVersion: aliases=[], creation_timestamp=1745050106210, current_stage='Production', description='0.9769543973941368', last_updated_timestamp=1745050106279, name='fake-news-model', run_id='9b82918f9f3842a989fef8930c7fd016', run_link='', source='s3://fake-news-prediction/4/9b82918f9f3842a989fef8930c7fd016/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>,
 <ModelVersion: aliases=[], creation_timestamp=1745049916172, current_stage='Archived', description='0.9561074918566775', last_updated_timestamp=1745050106235, name='fake-news-model', run_id='44139c0cd6e84b7e8ba09700e0f8c639', run_link='', source='s3://fake-news-prediction/4/44139c0cd6e84b7e8ba09700e0f8c639/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>]

In [2]:
for v in versions:
    if v.current_stage != "Archived":
        client.transition_model_version_stage(
            name=model_name,
            version=v.version,
            stage="Archived"
        )
    client.delete_model_version(name=model_name, version=v.version)

# Jetzt kannst du das registrierte Modell löschen
client.delete_registered_model(name=model_name)

/tmp/ipykernel_47024/870365967.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [42]:
import joblib
import mlflow
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

model_name = "fake-news-model"
version = client.get_model_version_by_alias(name=model_name, alias="Production")
RUN_ID = version.run_id

logged_model = f'runs:/{RUN_ID}/models'
model = mlflow.pyfunc.load_model(logged_model)

local_path = mlflow.artifacts.download_artifacts(run_id=RUN_ID, artifact_path="preprocessor.joblib")
preprocessor = joblib.load(local_path)

In [5]:
import pandas as pd

news = {
    "title": ' In RARE Case Of Factual Reporting, Fox Shows How Cruz Is Too Stupid To Run For President (VIDEO)',
    "text": 'For a party so terribly interested in governing this country, Republicans sure do lack the basic knowledge about how governing actually works. One of the latest examples of this is Ted Cruz and his choice to pick a vice presidential running mate at this stage in the process. In an incredibly rare case of factual reporting, Fox News explained in detail exactly why Cruz completely screwed up by doing this in the video below.Cruz made a huge deal out of the fact he picked a VP. He had a press conference just to announce it. Signs were made, people were cheering and then it turned out to be   Carly Fiorina? Her low popularity makes her a strange choice, but when you re Ted Cruz it s very likely you can t get a lot of the more popular people to hang out with you.Unpopularity aside, Cruz s pick of a VP is actually pretty ignorant for another reason   and this has nothing to do with Carly the Wicked Stepmother. There s virtually no chance that Ted Cruz will get the nomination outright. He would come up about 100 delegates short, even if he won every single one of them from here on out. This means that if Trump doesn t win outright (which he still can), the only hope Cruz has is a contested convention.That s where things completely go wrong for Cruz. Convention rules clearly state that a nominee cannot pick their own VP in a contested convention. The VP pick is up to the delegates, and if one were to make an educated guess, Carly Fiorina would NOT make the cut.Carly Fiorina s popularity is non-existent. Her entire campaign only lasted for two states, and then she was out. Convention delegates are extremely unlikely to pick a VP who they feel would be damaging to the party s chances of winning in November. So, this begs the question, did Ted even bother to read up on the rules of how you run a campaign for the presidency? Imagine his embarrassment after all the hoopla surrounding his pick, only to have her booted at the RNC.Watch Fox News explain how Ted Cruz has no clue how running for president works:Featured image via Scott Olson/Getty Images'
}

type(news)

pd.DataFrame([news])

,title,text
0,"In RARE Case Of Factual Reporting, Fox Shows ...",For a party so terribly interested in governin...


In [7]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from utils.utilityFunctions import prepare_features, apply_text_cleaner, load_file_s3

In [10]:
df = load_file_s3('fake-news-prediction', 'datasets/X_train_2025-04-08.parquet', 'parquet')
df.head()

,text_word_count,title_word_count,text_unique_words,text_char_count,title_char_count,avg_word_length,sentence_count,special_char_count,sentiment,title_text_clean
0,218,10,145,1059,53,4.857798,11,29,0.010000,prosecutor link suspect arrest last week near ...
1,182,11,123,1012,72,5.560440,9,37,-0.085714,breaking nyc protest get ugly anti american pr...
2,333,9,211,1670,65,5.015015,16,39,-0.032335,former republican senator endorses hillary cl...
3,308,10,211,1689,53,5.483766,21,56,0.082809,urge trump expand obamacare ensure healthcare ...
4,1430,18,635,6821,72,4.769930,62,228,0.050010,late test white house pull easter egg roll new...


In [11]:
X_val = load_file_s3('fake-news-prediction', 'datasets/X_val.parquet', 'parquet')
X_example = X_val[['title', 'text']]
X_example = X_example[:100]

In [15]:
X_val.loc[3,'title']

'Evangelical Jerry Falwell Jr. to head Trump education task force'

In [43]:
X = preprocessor.transform(X_example)
preds = model.predict(X)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [44]:
preds

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [39]:
import pandas as pd
X_example = prepare_features(X_example)
X_example = apply_text_cleaner(X_example, column='title_text')

Progress: 100%|███████████████████████████████████████████| 100/100 [00:08<00:00, 11.20it/s]


In [40]:
X_example.head()

,text_word_count,title_word_count,text_unique_words,text_char_count,title_char_count,avg_word_length,sentence_count,special_char_count,sentiment,title_text_clean
0,376,18,217,1693,79,4.502660,20,37,0.167725,rare case factual reporting fox show cruz stu...
1,998,15,438,4949,60,4.958918,62,137,0.095573,trump ’s obamacare promise hard keep new york ...
2,396,13,238,1958,72,4.944444,18,42,0.076924,nevertrump gopers demand emergency rnc meetin...
3,332,10,207,1812,55,5.457831,17,50,0.206944,evangelical jerry falwell jr head trump educat...
4,815,9,453,4387,51,5.325153,63,239,0.063842,surprise donald trump wrong immigrant crime do...


In [ ]:
# import pickle

# from flask import Flask, request, jsonify

import mlflow
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

model_name = "fake-news-model"
version = client.get_model_version_by_alias(name=model_name, alias="Production")
RUN_ID = version.run_id

logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)


def prepare_features(ride):
    features = {}
    features['PU_DO'] = '%s_%s' % (ride['PULocationID'], ride['DOLocationID'])
    features['trip_distance'] = ride['trip_distance']
    return features

def predict(features):
    preds = model.predict(features)
    return float(preds[0])

app = Flask('duration-prediction')

@app.route('/predict', methods=['POST'])
def predict_endpoint():
    ride = request.get_json()

    features = prepare_features(ride)
    pred= predict(features)

    result = {
        'duration': pred,
        'model_version': RUN_ID
    }

    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)